In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [7]:
save_dir = "/cluster/home/kamara/conceptx"
model_name = "gemma-3-4b"#"gpt4o-mini"#"mistral-7b-it"
dataset = "alpaca"

In [8]:
folder_path = os.path.join(save_dir, f"results-no-sample/explanations/{model_name}/{dataset}")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print("parts: ", parts)
        print("full_path:", full_path)
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        df["explainer"] = parts[dataset_idx + 2]
        
        if len(parts) > dataset_idx + 4:
            df["baseline"] = parts[dataset_idx + 3]
            seed_idx = dataset_idx + 4
        else:
            df["baseline"] = None
            seed_idx = dataset_idx + 3

        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)
df_all


parts:  ['explanations', 'batch', '0', 'alpaca', 'gemma-3-4b', 'random', '0.csv']
full_path: /cluster/home/kamara/conceptx/results-no-sample/explanations/gemma-3-4b/alpaca/random/seed_0/explanations_batch_0_alpaca_gemma-3-4b_random_0.csv
parts:  ['explanations', 'batch', '5', 'alpaca', 'gemma-3-4b', 'random', '0.csv']
full_path: /cluster/home/kamara/conceptx/results-no-sample/explanations/gemma-3-4b/alpaca/random/seed_0/explanations_batch_5_alpaca_gemma-3-4b_random_0.csv
parts:  ['explanations', 'batch', '1', 'alpaca', 'gemma-3-4b', 'random', '0.csv']
full_path: /cluster/home/kamara/conceptx/results-no-sample/explanations/gemma-3-4b/alpaca/random/seed_0/explanations_batch_1_alpaca_gemma-3-4b_random_0.csv
parts:  ['explanations', 'batch', '4', 'alpaca', 'gemma-3-4b', 'random', '0.csv']
full_path: /cluster/home/kamara/conceptx/results-no-sample/explanations/gemma-3-4b/alpaca/random/seed_0/explanations_batch_4_alpaca_gemma-3-4b_random_0.csv
parts:  ['explanations', 'batch', '2', 'alpaca',

,id,input,response,explanation,batch,dataset,model,explainer,baseline,seed
0,47316,What are the four rules for exponents?,"The four rules for exponents are: first, when ...","{'What_0': 1.0, 'are_1': 0.8523199056630058, '...",0,alpaca,gemma-3-4b,random,None,0
1,27527,How does the temperature affect the speed of s...,The temperature directly affects the speed of ...,"{'How_0': 0.08030088084084762, 'does_1': 0.343...",0,alpaca,gemma-3-4b,random,None,0
2,19941,Explain the process of mitosis in 200 words.,Mitosis is a fundamental process of cell divis...,"{'Explain_0': 0.891413761639218, 'the_1': 1.0,...",0,alpaca,gemma-3-4b,random,None,0
3,24644,Describe the events leading up to World War I.,The events leading up to World War I were a co...,"{'Describe_0': 0.42896326834685916, 'the_1': 0...",0,alpaca,gemma-3-4b,random,None,0
4,423,How does the human brain remember information?,The human brain remembers information through ...,"{'How_0': 0.9755586210641369, 'does_1': 0.6594...",0,alpaca,gemma-3-4b,random,None,0
...,...,...,...,...,...,...,...,...,...,...
11682,36651,Compose an email to invite your friends to a p...,Subject: You're Invited to a Party!\n\n ...,"{'Compose_0': np.float32(0.20065984), 'an_1': ...",9,alpaca,gemma-3-4b,conceptx,None,2
11683,18222,Summarize how technology has impacted education.,Technology has fundamentally reshaped educatio...,"{'Summarize_0': np.float32(0.29697847), 'how_1...",9,alpaca,gemma-3-4b,conceptx,None,2
11684,20622,Brainstorm ways to reduce energy consumption i...,"First, switch to LED light bulbs, as they use ...","{'Brainstorm_0': np.float32(0.30967614), 'ways...",9,alpaca,gemma-3-4b,conceptx,None,2
11685,12960,Compose a news article about the latest tech a...,Here's a news article about the latest tech ad...,"{'Compose_0': np.float32(0.25048533), 'a_1': n...",9,alpaca,gemma-3-4b,conceptx,None,2


In [9]:
### Size of dataset for model
model_list = ["llama-3-3b", "gemma-3-4b", "gpt4o-mini", "mistral-7b-it"]
df = df_all[df_all["model"].isin(model_list)]
df_stats = df.groupby(["dataset", "model", "seed", "explainer"])["id"].nunique().reset_index()

In [10]:
df_stats[df_stats['dataset']==dataset]

,dataset,model,seed,explainer,id
0,alpaca,gemma-3-4b,0,conceptshap,930
1,alpaca,gemma-3-4b,0,conceptx,927
2,alpaca,gemma-3-4b,0,random,1000
3,alpaca,gemma-3-4b,0,tokenshap,945
4,alpaca,gemma-3-4b,1,conceptshap,998
5,alpaca,gemma-3-4b,1,conceptx,996
6,alpaca,gemma-3-4b,1,random,1000
7,alpaca,gemma-3-4b,1,tokenshap,949
8,alpaca,gemma-3-4b,2,conceptshap,996
9,alpaca,gemma-3-4b,2,conceptx,991


In [11]:
df = df_all[df_all["model"].isin(model_list)]
df_stats2 = df.groupby(["dataset", "batch", "model", "seed", "explainer"])["id"].nunique().reset_index()
df_explore = df_stats2[df_stats2['dataset']==dataset]
for i in range(10):
    print(df_explore[df_explore["batch"]==str(i)])

   dataset batch       model seed    explainer   id
0   alpaca     0  gemma-3-4b    0  conceptshap  100
1   alpaca     0  gemma-3-4b    0     conceptx  100
2   alpaca     0  gemma-3-4b    0       random  100
3   alpaca     0  gemma-3-4b    0    tokenshap  100
4   alpaca     0  gemma-3-4b    1  conceptshap  100
5   alpaca     0  gemma-3-4b    1     conceptx   99
6   alpaca     0  gemma-3-4b    1       random  100
7   alpaca     0  gemma-3-4b    1    tokenshap  100
8   alpaca     0  gemma-3-4b    2  conceptshap  100
9   alpaca     0  gemma-3-4b    2     conceptx   99
10  alpaca     0  gemma-3-4b    2       random  100
11  alpaca     0  gemma-3-4b    2    tokenshap  100
   dataset batch       model seed    explainer   id
12  alpaca     1  gemma-3-4b    0  conceptshap  100
13  alpaca     1  gemma-3-4b    0     conceptx   99
14  alpaca     1  gemma-3-4b    0       random  100
15  alpaca     1  gemma-3-4b    0    tokenshap   95
16  alpaca     1  gemma-3-4b    1  conceptshap   98
17  alpaca  

In [30]:
def get_entropy(explanation_scores):
    positive_scores = np.array([v for v in explanation_scores.values() if v > 0])

    if positive_scores.sum() > 0:
        prob_distribution = positive_scores / positive_scores.sum()
        return -np.sum(prob_distribution * np.log2(prob_distribution))  # Use np.log2()
    else:
        print("No positive scores found, entropy is undefined.")
        return None

In [31]:
results = []
for _, row in df_all.iterrows():
    entry = row
    explanation = eval(row["explanation"], {"np": np, "nan": np.nan})
    contains_nan = any(np.isnan(value) for value in explanation.values())
    if contains_nan:
        continue
    entry['entropy'] = get_entropy(explanation)
    results.append(entry)

res_entropy = pd.DataFrame(results)


In [33]:
entropy_stats = res_entropy.groupby(['dataset', 'model', 'explainer'])['entropy'].agg(['mean', 'var']).reset_index()
#entropy_stats.to_csv('/cluster/home/kamara/conceptx/results/entropy/entropy_stats.csv')

In [34]:
entropy_stats

,dataset,model,explainer,mean,var
0,alpaca,mistral-7b-it,conceptshap,1.398062,0.277576
1,alpaca,mistral-7b-it,conceptx,1.385498,0.280361
2,alpaca,mistral-7b-it,random,2.469384,0.186801
3,alpaca,mistral-7b-it,tokenshap,2.385273,0.246139
4,genderbias,gemma-3-4b,conceptshap,1.598878,0.123187
5,genderbias,gemma-3-4b,conceptx,1.587445,0.127832
6,genderbias,gemma-3-4b,conceptx-a,1.641409,0.122830
7,genderbias,gemma-3-4b,conceptx-r,1.627461,0.128650
8,genderbias,gemma-3-4b,random,3.070639,0.064218
9,genderbias,gemma-3-4b,tokenshap,3.022261,0.081398


In [37]:
df = res_entropy[res_entropy["dataset"].isin(["alpaca", "genderbias", "sst2", "saladbench"])]
df_stats = df.groupby(['dataset', 'explainer'])['entropy'].agg(['mean', 'var']).reset_index()
df_stats = df_stats.pivot(index='explainer', columns='dataset', values=['mean', 'var'])

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "conceptx-a", "conceptx-r"]
explainer_map = {
    'conceptx': 'ConceptX',
    'conceptx-r': 'ConceptX-R',
    'conceptx-a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random'
}
df_stats = df_stats.rename(index=explainer_map)
df_stats = df_stats.reindex([explainer_map.get(e, e) for e in desired_order])

df_stats.columns = df_stats.columns.swaplevel()
df_stats = df_stats.sort_index(axis=1)
df_stats


dataset        alpaca           genderbias           saladbench            \
                 mean       var       mean       var       mean       var   
explainer                                                                   
Random       2.469384  0.186801   3.071507  0.063798   2.652021  0.228660   
TokenSHAP    2.385273  0.246139   3.032604  0.082337   2.592856  0.296397   
ConceptSHAP  1.398062  0.277576   1.603380  0.127844   1.050096  0.364592   
ConceptX     1.385498  0.280361   1.605361  0.121035   1.048883  0.361981   
ConceptX-A        NaN       NaN   1.632768  0.128408   1.081216  0.372281   
ConceptX-R        NaN       NaN   1.639094  0.120642        NaN       NaN   

dataset          sst2            
                 mean       var  
explainer                        
Random       2.198304  0.273436  
TokenSHAP    2.189302  0.309685  
ConceptSHAP  1.111496  0.371179  
ConceptX     1.163515  0.399792  
ConceptX-A   1.115665  0.361748  
ConceptX-R        NaN       NaN

In [29]:
model_name = "llama-3-3b"

df = res_entropy[res_entropy["dataset"].isin(["alpaca", "genderbias", "sst2", "sentiment"])]
df = df[df["model"]==model_name]
df_stats = df.groupby(['dataset', 'explainer'])['entropy'].agg(['mean', 'var']).reset_index()
df_stats = df_stats.pivot(index='explainer', columns='dataset', values=['mean', 'var'])

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "conceptx_a", "conceptx_r"]
explainer_map = {
    'conceptx': 'ConceptX',
    'conceptx_r': 'ConceptX-R',
    'conceptx_a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random'
}
df_stats = df_stats.rename(index=explainer_map)
df_stats = df_stats.reindex([explainer_map.get(e, e) for e in desired_order])

df_stats.columns = df_stats.columns.swaplevel()
df_stats = df_stats.sort_index(axis=1)
df_stats


dataset        alpaca           genderbias           sentiment            \
                 mean       var       mean       var      mean       var   
explainer                                                                  
Random       2.430585  0.197053   3.061705  0.059906  2.406416  0.183897   
TokenSHAP    2.417185  0.242259   3.099459  0.063215  2.438973  0.185631   
ConceptSHAP  1.363042  0.294692   1.602531  0.127037  1.150912  0.504957   
ConceptX     1.357986  0.290758   1.626354  0.115470  1.132997  0.495352   
ConceptX-A        NaN       NaN   1.640923  0.110102  1.146974  0.497077   
ConceptX-R        NaN       NaN   1.616368  0.135930       NaN       NaN   

dataset          sst2            
                 mean       var  
explainer                        
Random       2.209804  0.262754  
TokenSHAP    2.251246  0.274327  
ConceptSHAP  0.973283  0.391134  
ConceptX     0.972029  0.387504  
ConceptX-A   0.990884  0.386756  
ConceptX-R        NaN       NaN